In [ ]:
import pickle
file = open("pickle/1MD/1MD_imputation.pkl",'rb')
dataset = pickle.load(file)

In [ ]:
# ----------  init features selection----------
from ProjectML.monthDeath.pre_processing import *
from ProjectML.monthDeath.feature_extraction import *
X, y, dataset = extract_feature(dataset)
X = drop_corr_feature(X, 0.6)
X = best_eight_features(X)
# ----------  end features selection----------


In [ ]:
# load model
file = open("pickle/1MD/model_xgb.pkl",'rb')
xgb = pickle.load(file)


In [ ]:
# evaluation
from ProjectML.general_util.evaluation import *
print("Cross validation cv = 5 ")
balanced_accuracy_score = get_balanced_accuracy(X, y, xgb)
print("Balanced accuracy: %0.2f (+/- %0.2f)" % (balanced_accuracy_score.mean(), balanced_accuracy_score.std() * 2))
f1_score = get_f1_scores(X, y, xgb)
print("f1_score: %0.2f (+/- %0.2f)" % (f1_score.mean(), f1_score.std() * 2))

